# data

In [2]:
import config as cf
from util import load_label, load_data

print("Loading train and valid labels...")
# train_label = load_label(cf.train_label, cf)
valid_label = load_label(cf.valid_label, cf)

print("Loading train and valid data...")
# train_data = load_data(cf.train_data, train_label, True, cf)
valid_data = load_data(cf.valid_data, valid_label, False, cf)

Loading train and valid labels...
Loading train and valid data...


In [3]:
print(len(valid_data))

7215


In [4]:
from util import get_vocab, get_classes

train_data = valid_data
train_label = valid_label

#collect vocab and classes
print("Collecting text vocab...")
vocabxid, idxvocab, _ = get_vocab(train_data, "text", "char", cf.word_minfreq)

print("Collecting time zone vocab...")
tzxid, _, _ = get_vocab(train_data, "timezone", "word", 0)

print("Collecting location vocab...")
locxid, _, _ = get_vocab(train_data, "location", "char", cf.word_minfreq)

print("Collecting description vocab...")
descxid, _, _ = get_vocab(train_data, "description", "char", cf.word_minfreq)

print("Collecting name vocab...")
namexid, _, _ = get_vocab(train_data, "name", "char", cf.word_minfreq)

print("Collecting class labels...")
classes = get_classes(train_data, train_label)

In [5]:
from util import clean_data

print("Converting text to ids...")
valid_len_x, valid_miss_y, valid_len_loc, valid_len_desc, valid_len_name = clean_data(valid_data, valid_label, \
    vocabxid, tzxid, locxid, descxid, namexid, classes, cf)

Converting text to ids...


In [6]:
print("Sorting data based on tweet length...")
valid_data = sorted(valid_data, key=lambda item: len(item["x"]))

Sorting data based on tweet length...


# model

In [7]:
import tensorflow as tf
from geo_model import TGP

#train model
with tf.Graph().as_default(), tf.Session() as sess:
    tf.set_random_seed(cf.seed)
    initializer = tf.contrib.layers.xavier_initializer()
    
    mtrains, mvalids = [], []
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        mtrains.append(TGP(is_training=True, vocab_size=len(idxvocab), num_steps=cf.bucket_sizes[0], \
            num_classes=len(classes), num_timezones=len(tzxid), loc_vsize=len(locxid), \
            desc_vsize=len(descxid), name_vsize=len(namexid), config=cf))
        
#     with tf.variable_scope("model", reuse=True, initializer=initializer):
#         if len(cf.bucket_sizes) > 1:
#             for b in cf.bucket_sizes[1:]:
#                 mtrains.append(TGP(is_training=True, vocab_size=len(idxvocab), num_steps=b, \
#                     num_classes=len(classes), num_timezones=len(tzxid), loc_vsize=len(locxid), \
#                     desc_vsize=len(descxid), name_vsize=len(namexid),  config=cf))
#         for b in cf.bucket_sizes:
#             mvalids.append(TGP(is_training=False, vocab_size=len(idxvocab), num_steps=b, \
#                 num_classes=len(classes), num_timezones=len(tzxid), loc_vsize=len(locxid), \
#                 desc_vsize=len(descxid), name_vsize=len(namexid), config=cf))

    tf.initialize_all_variables().run()

W0905 15:29:08.546841 4642477504 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0905 15:29:08.548924 4642477504 deprecation_wrapper.py:119] From /Users/jinmei/Softwares/twitter-deepgeo/geo_model.py:127: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0905 15:29:08.557450 4642477504 deprecation_wrapper.py:119] From /Users/jinmei/Softwares/twitter-deepgeo/geo_model.py:144: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0905 15:29:08.558362 4642477504 deprecation_wrapper.py:119] From /Users/jinmei/Softwares/twitter-deepgeo/geo_model.py:37: The name tf.get_variable is deprecat

AttributeError: module 'tensorflow._api.v1.nn' has no attribute 'rnn'

In [ ]:
!pip show pandas

In [ ]:
!pip install --upgrade dask